# DOB Permit Issuance Analysis

*Author: Jiacheng Chen*

*Date: Aug 23, 2023*

In [ ]:
import pkg_resources

installed_packages = [pkg.key for pkg in pkg_resources.working_set]
print(installed_packages)

## Data Preperation

**Read Raw Data**

In [ ]:
# Read in raw issuance dataset
import pandas as pd
from sodapy import Socrata

# Identify Opendata NYC Source Domain
client = Socrata("data.cityofnewyork.us","8rVKWjYb75ZgPpouoX7bmtWNL")

# Get Active Major Construction permits through SoQL query

results = client.get_all("ipu4-2q9a", where="job_type = 'NB' OR job_type = 'A1'") # 'results' is a json 
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
#results_df.to_csv('MajorConstructionDF.csv') # I exported the df to csv for eaiser reuse

>SoQL doesn't support the SUBSTR operator and the dates of the raw data come in as strings. What a pity!

**Filter `Current Active` Constructions**

In [ ]:
# Convert 'dates' in raw df to date type
from datetime import datetime
results_df['expiration_date'] = pd.to_datetime(results_df['expiration_date'], format='mixed', dayfirst=True,  errors='ignore')

In [ ]:
results_df.dtypes

In [ ]:
# Get Current Date
today = datetime.now()

# Get currently active constructions
activeConstruction_df = results_df[results_df['expiration_date'] >= today]

# Format all date columns
# extract date columns from the raw df
dateToChange = results_df.columns[results_df.columns.str.contains('date')].tolist() 

# Convert 'dates' in raw df to date type
activeConstruction_df[dateToChange] = activeConstruction_df[dateToChange].apply(pd.to_datetime)

In [ ]:
# Check missing values
activeConstruction_df.isna().sum()

## EDA

**Key Metrics Summary**

I chose the categorical variables that don't have significant amount of missing values to be key metrics, such as `job_type`, `permit_type`, `recidential`, `owner's business type`, `non profit`, and I also included `dates` in the table, along with `GIS` variables. 

In [ ]:
# Select key columns
activeConstruction_df_summary = activeConstruction_df[['borough','bin__', 'job__', 'job_type', 
                                                       'bldg_type', 'residential','self_cert',
                                                       'permit_type','permit_subtype','permittee_s_license_type',
                                                       'permit_status', 'filing_status', 'owner_s_business_type',
                                                       'permit_si_no',
                                                       'filing_date', 'issuance_date', 'expiration_date',
                                                       'job_start_date','non_profit','gis_latitude', 
                                                       'gis_longitude','gis_council_district']]

In [ ]:
# Check and remove duplicates entries that ate exact identical
activeConstruction_df_cleaned = activeConstruction_df_summary[~activeConstruction_df_summary.duplicated(keep='last')]

I noticed there are some entries that a job number matchs mutiple permits and ves versa, but they have different work types or permit types. Considering the complexity of construction works, I decided only remove the rows that are exactly identical. 

*Job Types and Boroughs*

In [ ]:
# Group by Borough and Job types, count the number of each job type. 
jobType_sumTable = pd.DataFrame(activeConstruction_df_cleaned.groupby(['borough', 'job_type'])['job__'].count()).reset_index()
jobType_sumTable.columns = ('Borough', 'Job Type', 'Count') # rename columns

# Visualize the Borough Jobtype table
import seaborn as sns
sns.catplot(data=jobType_sumTable, x='Borough', y='Count', hue='Job Type', kind="bar", legend=False)

# Add labels and title
plt.xlabel("Borough")
plt.xticks(rotation=45)
plt.ylabel("Number of Jobs")
plt.title("Job Types by Borough")
plt.legend()  # Show the legend
plt.tight_layout()

# Save the plot
plt.savefig("job_types_by_borough.png", dpi=300)

In [ ]:
# Pivot the table, transform Job Type to columns
jobType_sumTable = jobType_sumTable.pivot(index='Borough', columns='Job Type', values='Count').reset_index()
jobType_sumTable.columns = ['Borough', 'Job Type A1', 'Job Type NB'] # rename columns

# Calculate row and column total Jobs by Boroughs and by Job Types
jobType_sumTable['Total'] = jobType_sumTable['Job Type A1'] + jobType_sumTable['Job Type NB'] # row total
total_row = jobType_sumTable[["Job Type A1", "Job Type NB", "Total"]].sum() # column total
total_row['Borough'] = 'Total'
jobType_sumTable = pd.concat([jobType_sumTable, pd.DataFrame([total_row])], ignore_index=True) # append column total to the table

# Save the Borough_JobType table to excel
import openpyxl
jobType_sumTable.to_excel("job_types_by_borough.xlsx")

In [ ]:
jobType_sumTable

*Permit Types and Boroughs*

In [ ]:
# Group by Borough and Job types, count the number of each job type. 
permitType_sumTable = pd.DataFrame(activeConstruction_df_cleaned.groupby(['borough', 'permit_type'])['job__'].count()).reset_index()
permitType_sumTable.columns = ('Borough', 'Permit Type', 'Count')

# Visualize the Permit Jobtype table
import seaborn as sns
sns.catplot(data=permitType_sumTable, x='Borough', y='Count', hue='Permit Type', kind="bar", legend=False)

# Add labels and title
plt.xlabel("Borough")
plt.xticks(rotation=45)
plt.ylabel("Number of Permit")
plt.title("Permit Types by Borough")
plt.legend()
plt.tight_layout()

# Save the plot
plt.savefig("permit_types_by_borough.png", dpi=300)

In [ ]:
# Pivot the table, transform Permit Type to columns
permitType_sumTable = permitType_sumTable.pivot(index='Borough', columns='Permit Type', values='Count').reset_index()

#replace missing value to 0
permitType_sumTable.fillna(0, inplace=True)

# Calculate row and column total Jobs by Boroughs and by Permit Types
permitType_sumTable['Total'] = permitType_sumTable.iloc[:, 1:].sum(axis=1)# row total
total_row = permitType_sumTable.iloc[:, 1:].sum() # column total
total_row['Borough'] = 'Total'
permitType_sumTable = pd.concat([permitType_sumTable, pd.DataFrame([total_row])], ignore_index=True) # append column total to table

# Save the Borough_JobType table to excel
permitType_sumTable.to_excel("permit_types_by_borough.xlsx")

In [ ]:
permitType_sumTable

**When the currently active major construction permits issued**

In [ ]:
# Numbers of Permit Issuance per day in each borough
TS_issuDate_Boro = activeConstruction_df_cleaned[['borough', 'issuance_date','job__']]
TS_issuDate_Boro = TS_issuDate_Boro.groupby(['borough','issuance_date'])['job__'].count().reset_index(name='issuance_count')

In [ ]:
# Time Series Plot
import matplotlib.pyplot as plt # I used matplotlin here cuz it's eaiser to resample the df by different time windows

TS_issuDate_Boro.index = TS_issuDate_Boro['issuance_date']
unique_boroughs = TS_issuDate_Boro['borough'].unique()

plt.figure(figsize=(10, 6))

# Loop through each borough to plot its time series data
for borough in unique_boroughs:
    data = TS_issuDate_Boro[TS_issuDate_Boro['borough'] == borough]['issuance_count']
    data_months = data.resample('M').sum() # Resample to get monthly data
    plt.plot(data_months, label=borough)

# Plot Settings
plt.xlabel('Month')
plt.ylabel('Number of Issuances')
plt.title('Current Active Major Constructions \n Permit Issuances Amount by Borough (Monthly)')
plt.legend()
plt.grid(True)
plt.xlim(pd.Timestamp('2022-01-01'), datetime.now())
plt.xticks(rotation=45) 
plt.savefig('TS_issuanceAmount_boro_monthly.png', dpi = 300)

plt.show()

Most currently active major construction permits were issued within **one year**, with some work begun as early as **January 1, 2022**. 

**Brooklyn** has the highest number of major ongoing construction projects. Notably, the issuance permits in Brooklyn has a peak in January 2023.

There are a few **outliers** that have been under construction since 2006.

In [ ]:
# extra long constructions 
# outliars
activeConstruction_df_cleaned[activeConstruction_df_cleaned['issuance_date']<'2022-01-01']

In [ ]:
import plotly.express as px
fig = px.line(pxtest, x='issuance_date', y='count',color='borough')
fig.update_xaxes(ticks= "outside",
                 ticklabelmode= "period", 
                 tickcolor= "black", 
                 ticklen=10, 
                 minor=dict(
                     ticklen=4,  
                     dtick=7*24*60*60*1000,  
                     tick0="2016-07-03", 
                     griddash='dot', 
                     gridcolor='white'),
                 rangeslider_visible=True,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1m", step="month", stepmode="backward"),
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="YTD", step="year", stepmode="todate"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(step="all")])
                     )
                )
fig.show()

# Code based on the example from Plotly at https://plotly.com/python/time-series/#adding-minor-ticks

Here is an interactive line plot that provides a comprehensive view of when the permits of active major constructions issued. This includes projects that were issued permits before January 1, 2022. Data points after today indicate projects that are currently undergoing the permit issuance process.

**Active major constrction sites**

In [ ]:
import geopandas as gpd

# Convert the working df to gdf
activeConstruction_gdf = gpd.GeoDataFrame(activeConstruction_df_cleaned, 
                                          geometry=gpd.points_from_xy(activeConstruction_df_cleaned.gis_longitude, activeConstruction_df_cleaned.gis_latitude))
# Set crs to 4326
activeConstruction_gdf.crs = 4326

In [ ]:
# Plot the points
activeConstruction_gdf.plot(alpha=0.1, markersize=50)

**Spatial Join with Business Improvement Districts**

BetaNYC has a very practical project that showcases various administrative boundaries in NYC. I found it would be interesting to explore Business Improvement Districts in context with major construction projects.

BetaNYC NYC Boundaries Map: https://beta.nyc/products/boundaries-map/

In [ ]:
# read in BID geojson
bid_url = "https://data.cityofnewyork.us/resource/7jdm-inj8.geojson"
bid_gdf = gpd.read_file(bid_url)

In [ ]:
bid_gdf.plot()

In [ ]:
# Add bid and active construction layers to map 
fig, bid_ac_ax= plt.subplots(figsize=(12,12))

# layer settings
bid_gdf.plot(alpha = 0.7, edgecolor = 'gray', color = 'green', ax=bid_ac_ax)
activeConstruction_gdf.plot(alpha = 0.5, markersize = 1, ax=bid_ac_ax)

# add a basemap
basemap = cx.providers.CartoDB.Positron
cx.add_basemap(bid_ac_ax, crs = 4326, source=basemap)

bid_ac_ax.axis('off')

In [ ]:
bid_gdf.head()

In [ ]:
# Spatial Join bid polygon geom to the point dataset
joined_layer_bid_ac = gpd.sjoin(activeConstruction_gdf, bid_gdf[['bid', 'geometry']], how="right", predicate='within')

# Group by and count number of points in bid polygons
joined_layer_bid_ac_count = gpd.GeoDataFrame(joined_layer_bid_ac.groupby(['bid','geometry']).size().reset_index(name = 'Count'))

In [ ]:
# plot the amount of active major constructions by bid
plt.figure(figsize=(12,12))

# layer setting
joined_bid_ac_ax = joined_layer_bid_ac_count.plot(column='Count', legend=True)

# add basemap
cx.add_basemap(joined_bid_ac_ax, crs = 4326, source=basemap)

# title and other settings
plt.title('Current Acvtive Major Constructions Count \n Business Improvement Districts ')
joined_bid_ac_ax.axis('off')

# save the output
plt.savefig('bid_activeMajorCons_map.png',dpi=300)

In [ ]:
# interactive map to navigate better
joined_layer_bid_ac_count.explore(column='Count', legend=True)

In [ ]:
# Identify Business Improvement Districts that have more than 20 active major constructions. 
joined_layer_bid_ac_count[joined_layer_bid_ac_count['Count']>=20].sort_values(by='Count')

In terms of New York City's business improvement districts, lower and midtown Manhattan are very busy with construction. Downtown Brooklyn has some as well. And I'm happy to see more constructions in LIC and Flushing because that suggests community vibrancy and business development. The increase in construction activity indicates positive growth and opportunity in these areas.

In [ ]:
# export the gdf to excel
joined_layer_bid_ac.to_excel('Bid_ActiveMajorContruction.xlsx', index = False)

In [ ]:
# convert date variables to string before exporting to shp file

# save date columns to a list
dateToStr = joined_layer_bid_ac.columns[joined_layer_bid_ac.columns.str.contains('date')].tolist() 
# for each column, convert datetime to date string
for column in dateToStr:
    joined_layer_bid_ac[column] = joined_layer_bid_ac[column].dt.strftime('%Y-%m-%d')

In [ ]:
# export layer to shp 
joined_layer_bid_ac.to_file('bid_ActiveMajorConstruction.shp')

## Spatial Autocorrelation

I decided to conduct a Spatial Aitocorrelation analysis to determine whether clusters of active construction projects exist. My goal is to identify where the clusters are and to compare them with the Business Improvement Districts map. To achieve this, I employed Moran's Index. 

**Prepare the basemap**

In [ ]:
import numpy as np
import shapely

def make_grid(gdf, n_cells):
    gdf = gdf.copy()
    xmin, ymin, xmax, ymax= gdf.total_bounds
    cell_size = (xmax-xmin)/n_cells
    # create the cells in a loop
    grid_cells = []
    for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
        for y0 in np.arange(ymin, ymax+cell_size, cell_size):
            x1 = x0-cell_size
            y1 = y0+cell_size
            grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
    grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=gdf.crs)
    return grid

# code based on lecture note from WIll Geary at https://github.com/willgeary/info615/blob/main/modules/09-spatial-point-patterns/notebook.ipynb 

In [ ]:
# genarate a grid over map
grid = make_grid(activeConstruction_gdf, n_cells=80)

# overlap the points with grid
ax = activeConstruction_gdf.plot(markersize=2)
grid.plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=1)
ax.axis('off');

In [ ]:
# to aggregate and summarize points over grid
def rasterize(gdf, grid, aggfunc="count", column=None, plot=True):
    merged = gpd.sjoin(gdf, grid, how='left', predicate='within').copy()
    if aggfunc == "count":
        column = 'count'
        output_col = column
        merged[column] = 1
    else:
        output_col = column + "_" + aggfunc   
    dissolved = merged.dissolve(by="index_right", aggfunc=aggfunc)[[column]]
    dissolved.columns = [output_col]
    grid.loc[dissolved.index, output_col] = dissolved[output_col].values
    if plot:
        ax = grid.plot(column=output_col, figsize=(12, 8), edgecolor="grey", legend=True)
        ax.axis('off')
        cx.add_basemap(ax,source=cx.providers.CartoDB.Positron,crs=gdf.crs)
        plt.show()
    return grid

In [ ]:
r = rasterize(activeConstruction_gdf, grid, aggfunc="count", plot=True)

In [ ]:
# drop grid cells contain no points
r = r.dropna(subset=['count'])

**Global Moran's I**

In [ ]:
# build a spatial weights matrix
from pysal.lib import weights

# Use Queen contiguity to generate weights
w = weights.Queen.from_dataframe(r)
# row-standardized weight matrix
w.transform = 'R'

w.weights

In [ ]:
# Global spatial autocorrelation hypothesis test
from pysal.explore import esda

# H0: There is no spatial autocorrelation 
# Ha: There no spatial autocorrelation 
moran = esda.moran.Moran(r['count'], w, permutations=9999)

In [ ]:
moran.I

In [ ]:
moran.p_sim

The simulated p-value is very low. We are confident to reject the null hypothesis and conclude that spatial autocorrelation is in fact present. 

In [ ]:
# plot the Global Moran's I results
from splot.esda import plot_moran
plot_moran(moran)

**Local Moran's I**

In [ ]:
# test for local spatial autocorrelation
local_moran = esda.moran.Moran_Local(r['count'], w) # use the same wright matrix

In [ ]:
# plot local moran's I results
from splot.esda import plot_local_autocorrelation
plot_local_autocorrelation(local_moran, r, 'count', p=0.05)

**Compare with Business Improvement Districts**

In [ ]:
r['local_moran_Is'] = local_moran.Is
r['local_moran_p_value'] = local_moran.p_sim
r['local_moran_quadrant'] = local_moran.q

alpha = 0.05
hotspots = r.query(f"local_moran_p_value < {alpha} & local_moran_quadrant == 1")
coldspots = r.query(f"local_moran_p_value < {alpha} & local_moran_quadrant == 3")
doughnuts = r.query(f"local_moran_p_value < {alpha} & local_moran_quadrant == 2")
diamonds = r.query(f"local_moran_p_value < {alpha} & local_moran_quadrant == 4")

In [ ]:
fig, cluster_map = plt.subplots(figsize = (16,8))
r.plot(ax=cluster_map, facecolor='none', alpha=0.05)
hotspots.plot(color='red', ax=cluster_map, label='Hot Spot')
coldspots.plot(color='blue', ax=cluster_map, label='Cold Spot')
doughnuts.plot(color='blue', ax=cluster_map, label='Doughnuts', alpha=0.5, edgecolor='blue')
diamonds.plot(color='red', ax=cluster_map, label='Diamonds',alpha=0.5, edgecolor='red')

joined_layer_bid_ac_count.plot(column='Count', legend=True, ax=cluster_map)

The tessellation grids appear to be too large compare to the bid boundries. However, we can generally tell there are overlaps between the hot spots and the business improvement districts.